In [ ]:
from config import CLIENT_ID, CLIENT_SECRET
import os
import pprint
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
playlist_df = pd.read_csv(f'./Resources/playlists.csv')

In [ ]:
playlist_df = playlist_df.dropna(axis=0)

In [ ]:
playlist_df

In [ ]:
# Set environment variables
os.environ['SPOTIPY_CLIENT_ID'] = CLIENT_ID
os.environ['SPOTIPY_CLIENT_SECRET'] = CLIENT_SECRET

In [ ]:
# instantiate Spotify class
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

In [ ]:
#playlist_dfs = []
songs_dfs = []

# Iterate over playlist dataframe to grab song info for each playlist
for index, row in playlist_df[:50].iterrows():
    
    print(f'Processing playlist {index} of {len(playlist_df)}')
    
    # Grabbing URI for the playlist
    playlist_uri = row['URI'] 
    print(playlist_uri)
    
    # Make temporary table for each playlist: subsetting playlist df
    temp = playlist_df[playlist_df['URI']==playlist_uri]
    
    # Get all songs from playlist
    song_count = len(sp.playlist_tracks(playlist_uri)["items"])
    
    # Temporary table for all songs in this playlist
    songs = []
    
    # Get info for each song
    for song_index in range(0, song_count):

        print(f'\t Adding info for song {song_index} out of {song_count}... \n')
        
        song_temp = temp.copy()
        
        try:
            ## Add song info to the table
            # Grab URI
            song_temp['Song URI'] = sp.playlist_tracks(playlist_uri)["items"][song_index]["track"]["uri"]

            # Grab Popularity
            song_temp['Popularity'] = sp.playlist_tracks(playlist_uri)["items"][song_index]["track"]["popularity"]

            # Grab Artist Name
            song_temp['Artist Name'] = sp.playlist_tracks(playlist_uri)["items"][song_index]["track"]["artists"][0]['name']

            # Grab Song Name
            song_temp['Song Name'] = sp.playlist_tracks(playlist_uri)["items"][song_index]["track"]["name"]

            # Grab popularity score
            songs.append(song_temp)

        except TypeError:
            print(f'\t Skipping info for song {song_index} out of {song_count}... \n')
            
    
    # Make song table
    songs = pd.concat(songs)
    
    # Add to playlist
    #playlist_dfs.append(songs)
    songs_dfs.append(songs)

# Bring everything together 
pd.concat(songs_dfs)
playlist_df = pd.DataFrame(pd.concat(songs_dfs))
#pd.concat(playlist_dfs)

## maybe there is a better way to do this but the following line isn't something that we had and therefore the dataframe I had exported previously only grabbed the last "songs_dfs" and was returned as a list instead of a dataframe

In [ ]:
playlist_df = pd.DataFrame(pd.concat(songs_dfs))

In [ ]:
playlist_df.head()

In [ ]:
playlist_df.to_csv('Resources/first50_playlists.csv')